## 言語処理100本ノック2020 in Haskell

### chapter 1

In [1]:
:extension OverloadedStrings
import qualified Data.Text as T
import Prelude as P

In [2]:
-- 00
src ="stressed" :: T.Text
putStrLn $ T.unpack $ T.reverse src 

desserts

In [3]:
-- 00
src = "パタトクカシーー" :: T.Text

f :: T.Text -> Maybe T.Text
f s | T.length s < 7 =  Nothing
    | otherwise = Just $ T.pack [T.head s, T.index s 2, T.index s 4, T.index s 6]

case f src of 
    Just s -> putStrLn $ T.unpack s
    Nothing -> return ()

パトカー

In [4]:
-- 00
src1 = "パトカー"
src2 = "タクシー"

f :: T.Text -> T.Text -> T.Text
f z w = P.foldr (\s t -> T.pack [fst s, snd s] <> t)  "" (T.zip z w) 

putStrLn $ T.unpack $ f src1 src2

パタトクカシーー

In [5]:
-- 00
src = "Now I need a drink, alcoholic of course, after the heavy lectures involving quantum mechanics."

getCountList :: T.Text -> [Int]
getCountList s = 
    let 
        ws' = P.map (T.filter (\c -> c /= ',' && c /= '.')) (T.words s)
    in 
        builder ws' []
    where 
        builder :: [T.Text] -> [Int] -> [Int]
        builder ws counts | P.null ws = counts
                          | otherwise = builder (P.tail ws) (counts <> [T.length (P.head ws)])

getCountList src

[3,1,4,1,5,9,2,6,5,3,5,8,9,7,9]

In [7]:
-- 00
import qualified Data.HashMap.Strict as HM
import Data.Function
src = "Hi He Lied Because Boron Could Not Oxidize Fluorine. New Nations Might Also Sign Peace Security Clause. Arthur King Can."
onlyOne = [0, 4, 5, 6, 7, 8, 14, 15, 18]

f :: T.Text -> T.Text
f s =
    let
        ws = T.words s
        wns = P.zip [0..(P.length ws -1)] ws
    in seq ws $
        if P.length ws < 19 then ""
        else seq wns $
            P.foldr (\s t -> let (ind, text) = s in if ind `elem` onlyOne then T.pack [T.head text] <> t else T.pack [T.head text, T.index text 1] <> t) "" wns

putStrLn $ T.unpack $ f src

pairingIndex :: T.Text -> HM.HashMap T.Text Int
pairingIndex sentences =
    let
        ws' = T.words src
        ws = P.filter ( /= ".") $! ws'
        wns = P.zip [0..(P.length ws -1)] ws
    in
        flip fix (HM.empty, wns) $ \loop (dict, rest) ->
            if P.null rest then dict
            else
                let (v, k) = P.head rest in
                loop (HM.insert k v dict, P.tail rest)


print $ pairingIndex src 


HHeLiBeBCNOFNeNaMiAlSiPSClArKCa

fromList [("Might",11),("Can.",19),("Not",6),("Nations",10),("Boron",4),("New",9),("Sign",13),("Clause.",16),("Fluorine.",8),("Could",5),("Oxidize",7),("Lied",2),("He",1),("Because",3),("Hi",0),("Also",12),("Peace",14),("Arthur",17),("King",18),("Security",15)]

In [ ]:
--0.5
getNGram :: Int -> T.Text -> [T.Text] -> [T.Text]
getNGram n s nGrams | n <= 0 = nGrams
                    | T.length s < n = nGrams
                    | otherwise = getNGram n (T.drop 1 s) (nGrams <> [T.take n s])

getNWordsGram :: Int -> T.Text -> [T.Text]
getNWordsGram n s = builder n (T.words s) []
    where
        builder :: Int -> [T.Text] -> [T.Text] -> [T.Text]
        builder n ws nGrams | P.length ws < n = nGrams
                            | n <= 0 = nGrams
                            | otherwise = builder n (P.drop 1 ws) (nGrams <> [T.concat (P.map (\t -> t <> " ") $ P.take n ws)])

print $ getNGram 2 "I am an NLPer" []
print $ getNWordsGram 2 "I am an NLPer"

Line 12: Avoid lambda using `infix`
Found:
(\ t -> t <> " ")
Why not:
(<> " ")

["I "," a","am","m "," a","an","n "," N","NL","LP","Pe","er"]

["I am ","am an ","an NLPer "]

In [ ]:
-- 00
import qualified Data.HashSet as HS
setX = HS.fromList $ getNGram 2 "paraparaparadise" []
setY = HS.fromList $ getNGram 2 "paragraph" []

print $ HS.unions [setX, setY]
print $ HS.intersection setX setY
print $ HS.difference setX setY

fromList ["ad","gr","ar","pa","di","ag","se","is","ph","ra","ap"]

fromList ["ar","pa","ra","ap"]

fromList ["ad","di","se","is"]

In [ ]:
-- 00
{-# Language ScopedTypeVariables #-}
{-# Language TypeApplications #-}
{-# Language ConstraintKinds #-}
{-# Language GADTs #-}

import qualified Type.Reflection as TRF

type Generable a = (Eq a, TRF.Typeable a, Show a)

packToText :: forall a. (Eq a, TRF.Typeable a, Show a) => a -> T.Text
packToText x =  
    case TRF.eqTypeRep (TRF.typeRep @a) (TRF.typeRep @String) of
        -- コンパイラは a と String が等しいことがわかる
        Just HRefl -> T.pack x
        Nothing -> T.pack $ show x

f :: (Generable a, Generable b, Generable c) => a -> b -> c -> T.Text
f x y z = packToText x <> "時の" <> packToText y <> "は" <> packToText z

putStrLn $ T.unpack $ f 12 "気温" 22.4


12時の気温は22.4

In [34]:
-- 00
import qualified Data.ByteString as BS
import qualified Data.ByteString.UTF8 as BSU
import Data.Char

encriptChar :: Char -> Char
encriptChar c | isLower c = 
    let 
        cb = head $ BS.unpack $ BSU.fromChar c
    in
        head $ BSU.toString $ BS.pack [219 - cb]
              | otherwise = c

decriptChar :: Char -> Char
decriptChar c =
    if check c then
        head $ BSU.toString $ BS.pack [219 - head (BS.unpack $ BSU.fromChar c)]
    else
        c
    where
        check c = 
            let
                target = head (BS.unpack $ BSU.fromChar c) 
                min = head $ BS.unpack $ BSU.fromChar 'a'
                max = head $ BS.unpack $ BSU.fromChar 'z'
            in 
                min <= target && target <= max


encript :: T.Text -> T.Text
encript = T.map encriptChar

decript :: T.Text -> T.Text
decript = T.map decriptChar

message = "こんにちは this is encripted message. THIS IS NOT ENCRIPTED"
putStr $ T.unpack $ encript message
putStr $ T.unpack $ decript (encript message) 


こんにちは gsrh rh vmxirkgvw nvhhztv. THIS IS NOT ENCRIPTED

こんにちは this is encripted message. THIS IS NOT ENCRIPTED

In [53]:
-- 00
import qualified System.Random as RD
import Debug.Trace

factorial :: Int -> Int
factorial 0 = 1
factorial n | n > 0 = let n' = factorial (n -1) in seq n' $ n * n' 

interleave :: a -> [a] -> [[a]]
interleave x [] = [[x]]
interleave x (y:ys) = (x:y:ys) : map (y:) (interleave x ys)

perms :: [a] -> [[a]]
perms = foldr (concatMap . interleave) [[]]

getTypoglycemia :: RD.RandomGen g => g -> String -> String
getTypoglycemia g t | length t > 4 = 
    let
        seed = perms (tail (init t))
        n = fst $ RD.randomR (0, factorial (length t - 2) -1) g
    in
        [head t] ++ (seed !! n) ++ [last t]
                    | otherwise = t

arg = "I couldn’t believe that I could actually understand what I was reading : the phenomenal power of the human mind ."

do 
    g <- RD.newStdGen
    putStrLn $ unwords $map (getTypoglycemia g) (words arg)


I cdoun’lt bvieele that I cuold aactlluy utnseadnrd what I was rndeiag : the pehmnneaol pwoer of the hmuan mind .

### chapter 2